# Summary

Here, we show the process using scTSS to identify TSS and using Brie2 to detect alternative TSS in NPC dataset. 



# Content

* run scTSS
  - merge bam file of NPC and NLH
  - run scTSS
* use Brie2 to detect alternative TSS usage
  - make h5ad file
  - get h5ad file of each cell type
  - get cdr file
  - run Brie2
* do filtering (FDR <0.01)


# run scTSS

## merge bam file of NPC and NLH

nohup samtools merge  NPC_NLH.bam -b bamlist.fofn  -@ 20 --write-index &

The bamlist.fofn looks like this:

/storage/yhhuang/users/ruiyan/NPC/srafiledir/NLH/NLH.bam
/storage/yhhuang/users/ruiyan/NPC/srafiledir/NPC/NPC.bam 

## run scTSS

#!/bin/bash

gtfFile=/storage/yhhuang/users/ruiyan/annotation/refdata-gex-GRCh38-2020-A/genes/genes.gtf
testgtfFile=/home/houruiyan/old_sctss/scTSS/test/Homo_sapiens.GRCh38.105.chr_test.gtf
fastaFile=/storage/yhhuang/users/ruiyan/annotation/refdata-gex-GRCh38-2020-A/fasta/genome.fa


scTSS-count --gtf $gtfFile --refFastq  $fastaFile  --bam /storage/yhhuang/users/ruiyan/NPC/srafiledir/scTSS_rerun/rawdata/NPC_NLH.bam  -c /storage/yhhuang/users/ruiyan/NPC/cdr_file/all_cell.tsv  -o /storage/yhhuang/users/ruiyan/NPC/srafiledir/scTSS_out/all_unannotation  --mode Unannotation --minCount 50  -p 20  --maxReadCount 10000 --clusterDistance 500

# use Brie2 to detect alternative TSS usage

## prepare file to run Brie2

### make h5ad file

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import scanpy as sc
import numpy as np

In [4]:
adata=sc.read('/storage3/yhhuang/users/ruiyan/development_data/processed_file/rerun_all_preprocessing_alllen_norm.h5ad')
adata

AnnData object with n_obs × n_vars = 37559 × 5449
    obs: 'cell_id', 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    var: 'TSS_start', 'TSS_end', 'gene_id', 'Chromosome', 'Feature', 'Start', 'End', 'Strand', 'gene_name', 'len'
    uns: 'log1p'
    obsm: 'X_umap'
    layers: 'counts'

In [5]:
expadata=sc.read('/storage3/yhhuang/users/ruiyan/development_data/processed_file/cellranger_changeCellID/all_cellranger_development.h5ad')
expadata

AnnData object with n_obs × n_vars = 5399108 × 36601
    obs: 'index', 'cell_id', 'batch'
    var: 'gene_ids', 'feature_types'

In [6]:
expadata.var['gene_name']=expadata.var.index
expadata.var.index=expadata.var['gene_ids']

In [7]:
expadata.var

,gene_ids,feature_types,gene_name
gene_ids,,,
ENSG00000243485,ENSG00000243485,Gene Expression,MIR1302-2HG
ENSG00000237613,ENSG00000237613,Gene Expression,FAM138A
ENSG00000186092,ENSG00000186092,Gene Expression,OR4F5
ENSG00000238009,ENSG00000238009,Gene Expression,AL627309.1
ENSG00000239945,ENSG00000239945,Gene Expression,AL627309.3
...,...,...,...
ENSG00000277836,ENSG00000277836,Gene Expression,AC141272.1
ENSG00000278633,ENSG00000278633,Gene Expression,AC023491.2
ENSG00000276017,ENSG00000276017,Gene Expression,AC007325.1


In [8]:
expadata.obs_names_make_unique()

expadata

AnnData object with n_obs × n_vars = 5399108 × 36601
    obs: 'index', 'cell_id', 'batch'
    var: 'gene_ids', 'feature_types', 'gene_name'

In [9]:
genedf=adata.var.copy()
genedf

,TSS_start,TSS_end,gene_id,Chromosome,Feature,Start,End,Strand,gene_name,len
transcript_id,,,,,,,,,,
ENSG00000197530_newTSS,1629513,1629602,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196
ENSG00000197530_ENST00000487053,1615842,1615943,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196
ENSG00000197530_ENST00000518681,1615420,1615554,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196
ENSG00000197530_ENST00000510793,1616193,1616435,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196
ENSG00000189409_newTSS,1632164,1632227,ENSG00000189409,1,gene,1632162,1635263,+,MMP23B,3101
...,...,...,...,...,...,...,...,...,...,...
ENSG00000130830_newTSS,154799743,154799794,ENSG00000130830,X,gene,154778683,154821007,-,MPP1,42324
ENSG00000067048_ENST00000493363,12904785,12904950,ENSG00000067048,Y,gene,12904107,12920478,+,DDX3Y,16371
ENSG00000067048_ENST00000440554,12905701,12905772,ENSG00000067048,Y,gene,12904107,12920478,+,DDX3Y,16371


In [10]:
genedf['count']=np.sum(adata.X,axis=0)
genedf

,TSS_start,TSS_end,gene_id,Chromosome,Feature,Start,End,Strand,gene_name,len,count
transcript_id,,,,,,,,,,,
ENSG00000197530_newTSS,1629513,1629602,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196,256.473970
ENSG00000197530_ENST00000487053,1615842,1615943,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196,4405.169038
ENSG00000197530_ENST00000518681,1615420,1615554,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196,2676.125020
ENSG00000197530_ENST00000510793,1616193,1616435,ENSG00000197530,1,gene,1615414,1630610,+,MIB2,15196,412.545578
ENSG00000189409_newTSS,1632164,1632227,ENSG00000189409,1,gene,1632162,1635263,+,MMP23B,3101,1120.169594
...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000130830_newTSS,154799743,154799794,ENSG00000130830,X,gene,154778683,154821007,-,MPP1,42324,1864.098669
ENSG00000067048_ENST00000493363,12904785,12904950,ENSG00000067048,Y,gene,12904107,12920478,+,DDX3Y,16371,21090.087880
ENSG00000067048_ENST00000440554,12905701,12905772,ENSG00000067048,Y,gene,12904107,12920478,+,DDX3Y,16371,916.549825


In [11]:
keepdf=genedf.sort_values(['gene_id','count']).groupby('gene_id').tail(2)
keepdf

,TSS_start,TSS_end,gene_id,Chromosome,Feature,Start,End,Strand,gene_name,len,count
transcript_id,,,,,,,,,,,
ENSG00000000460_ENST00000472795,169794679,169794816,ENSG00000000460,1,gene,169662006,169854080,+,C1orf112,192074,511.390458
ENSG00000000460_ENST00000359326,169794982,169795207,ENSG00000000460,1,gene,169662006,169854080,+,C1orf112,192074,21252.589071
ENSG00000000938_newTSS,27635064,27635076,ENSG00000000938,1,gene,27612063,27635185,-,FGR,23122,201.604448
ENSG00000000938_ENST00000457296,27626061,27626162,ENSG00000000938,1,gene,27612063,27635185,-,FGR,23122,618.203238
ENSG00000001626_ENST00000647720,117592386,117592554,ENSG00000001626,7,gene,117465783,117715971,+,CFTR,250188,328.055856
...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285646_ENST00000650448,11908143,11908221,ENSG00000285646,1,gene,11907939,11914298,+,AL021155.5,6359,13139.479124
ENSG00000285756_ENST00000671583,3843689,3843850,ENSG00000285756,X,gene,3817527,3937855,-,BX890604.2,120328,4975.422838
ENSG00000285756_newTSS,3882165,3882287,ENSG00000285756,X,gene,3817527,3937855,-,BX890604.2,120328,6140.318460


In [12]:
firstdf=keepdf[::2]
firstdf

,TSS_start,TSS_end,gene_id,Chromosome,Feature,Start,End,Strand,gene_name,len,count
transcript_id,,,,,,,,,,,
ENSG00000000460_ENST00000472795,169794679,169794816,ENSG00000000460,1,gene,169662006,169854080,+,C1orf112,192074,511.390458
ENSG00000000938_newTSS,27635064,27635076,ENSG00000000938,1,gene,27612063,27635185,-,FGR,23122,201.604448
ENSG00000001626_ENST00000647720,117592386,117592554,ENSG00000001626,7,gene,117465783,117715971,+,CFTR,250188,328.055856
ENSG00000001631_newTSS,92245432,92245492,ENSG00000001631,7,gene,92198968,92246166,-,KRIT1,47198,1285.028807
ENSG00000002834_newTSS,38870152,38870251,ENSG00000002834,17,gene,38869858,38921770,+,LASP1,51912,269.558158
...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285043_ENST00000395248,30053091,30053222,ENSG00000285043,16,gene,30053089,30070420,+,AC093512.2,17331,1403.487265
ENSG00000285331_newTSS,56886908,56887106,ENSG00000285331,15,gene,56795459,56918499,-,AC090517.5,123040,612.365203
ENSG00000285646_newTSS,11909037,11909155,ENSG00000285646,1,gene,11907939,11914298,+,AL021155.5,6359,575.438952


In [13]:
seconddf=keepdf[1::2]
seconddf

,TSS_start,TSS_end,gene_id,Chromosome,Feature,Start,End,Strand,gene_name,len,count
transcript_id,,,,,,,,,,,
ENSG00000000460_ENST00000359326,169794982,169795207,ENSG00000000460,1,gene,169662006,169854080,+,C1orf112,192074,21252.589071
ENSG00000000938_ENST00000457296,27626061,27626162,ENSG00000000938,1,gene,27612063,27635185,-,FGR,23122,618.203238
ENSG00000001626_newTSS,117479966,117480057,ENSG00000001626,7,gene,117465783,117715971,+,CFTR,250188,796.037284
ENSG00000001631_ENST00000412043,92245654,92245871,ENSG00000001631,7,gene,92198968,92246166,-,KRIT1,47198,23266.858150
ENSG00000002834_ENST00000585841,38869959,38870142,ENSG00000002834,17,gene,38869858,38921770,+,LASP1,51912,29300.639149
...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285043_newTSS,30064056,30064226,ENSG00000285043,16,gene,30053089,30070420,+,AC093512.2,17331,22023.363957
ENSG00000285331_ENST00000648603,56918242,56918390,ENSG00000285331,15,gene,56795459,56918499,-,AC090517.5,123040,7929.171224
ENSG00000285646_ENST00000650448,11908143,11908221,ENSG00000285646,1,gene,11907939,11914298,+,AL021155.5,6359,13139.479124


In [14]:
keepexpadata=expadata[adata.obs.index.tolist(),seconddf['gene_id'].tolist()]
keepexpadata

View of AnnData object with n_obs × n_vars = 37559 × 2356
    obs: 'index', 'cell_id', 'batch'
    var: 'gene_ids', 'feature_types', 'gene_name'

In [15]:
keepexpadata.obs=adata.obs
keepexpadata

AnnData object with n_obs × n_vars = 37559 × 2356
    obs: 'cell_id', 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    var: 'gene_ids', 'feature_types', 'gene_name'

In [16]:
isoform1adata=adata[:,firstdf.index.tolist()]
isoform1adata

View of AnnData object with n_obs × n_vars = 37559 × 2356
    obs: 'cell_id', 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    var: 'TSS_start', 'TSS_end', 'gene_id', 'Chromosome', 'Feature', 'Start', 'End', 'Strand', 'gene_name', 'len'
    uns: 'log1p'
    obsm: 'X_umap'
    layers: 'counts'

In [17]:
isoform2adata=adata[:,seconddf.index.tolist()]
isoform2adata

View of AnnData object with n_obs × n_vars = 37559 × 2356
    obs: 'cell_id', 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    var: 'TSS_start', 'TSS_end', 'gene_id', 'Chromosome', 'Feature', 'Start', 'End', 'Strand', 'gene_name', 'len'
    uns: 'log1p'
    obsm: 'X_umap'
    layers: 'counts'

In [18]:
isoform2adata.var.index

Index(['ENSG00000000460_ENST00000359326', 'ENSG00000000938_ENST00000457296',
       'ENSG00000001626_newTSS', 'ENSG00000001631_ENST00000412043',
       'ENSG00000002834_ENST00000585841', 'ENSG00000002919_ENST00000578861',
       'ENSG00000002933_ENST00000004103', 'ENSG00000003402_ENST00000443227',
       'ENSG00000004534_ENST00000443081', 'ENSG00000004897_ENST00000531206',
       ...
       'ENSG00000278558_ENST00000618859', 'ENSG00000280594_newTSS',
       'ENSG00000283103_ENST00000658331', 'ENSG00000284634_ENST00000641608',
       'ENSG00000284770_ENST00000642463', 'ENSG00000285043_newTSS',
       'ENSG00000285331_ENST00000648603', 'ENSG00000285646_ENST00000650448',
       'ENSG00000285756_newTSS', 'ENSG00000286340_ENST00000669651'],
      dtype='object', name='transcript_id', length=2356)

In [19]:
keepexpadata.layers['isoform1']=isoform1adata.layers['counts']
keepexpadata.layers['isoform2']=isoform2adata.layers['counts']

In [20]:
keepexpadata.var['isoform1_name']=isoform1adata.var.index.values
keepexpadata.var['isoform2_name']=isoform2adata.var.index.values
keepexpadata

AnnData object with n_obs × n_vars = 37559 × 2356
    obs: 'cell_id', 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    var: 'gene_ids', 'feature_types', 'gene_name', 'isoform1_name', 'isoform2_name'
    layers: 'isoform1', 'isoform2'

In [21]:
keepexpadata.layers['ambiguous']=np.zeros((37559, 2356))
keepexpadata

AnnData object with n_obs × n_vars = 37559 × 2356
    obs: 'cell_id', 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    var: 'gene_ids', 'feature_types', 'gene_name', 'isoform1_name', 'isoform2_name'
    layers: 'isoform1', 'isoform2', 'ambiguous'

In [22]:
keepexpadata.X=keepexpadata.X.astype('float32')

In [23]:
keepexpadata.layers['isoform1']=keepexpadata.layers['isoform1'].astype('float32')
keepexpadata.layers['isoform2']=keepexpadata.layers['isoform2'].astype('float32')
keepexpadata.layers['ambiguous']=keepexpadata.layers['ambiguous'].astype('float32')

keepexpadata

AnnData object with n_obs × n_vars = 37559 × 2356
    obs: 'cell_id', 'Anno_level_1', 'Anno_level_2', 'Anno_level_3', 'Anno_level_4', 'Anno_level_5', 'Anno_level_fig1', 'Sample', 'donor', 'organ', 'sort', 'file', 'Anno_stage', 'Age', 'Gender', 'Source'
    var: 'gene_ids', 'feature_types', 'gene_name', 'isoform1_name', 'isoform2_name'
    layers: 'isoform1', 'isoform2', 'ambiguous'

In [24]:
keepexpadata.X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [ ]:
keepexpadata.write('/storage3/yhhuang/users/ruiyan/development_data/Brie/express_to_brie.h5ad')

### Get h5ad file of cell type

In [25]:
import pandas as pd
import scanpy as sc
import numpy as np
import os

In [26]:
adata=sc.read('/storage/yhhuang/users/ruiyan/NPC/express_to_brie.h5ad')
adata

AnnData object with n_obs × n_vars = 51001 × 1784
    obs: 'cluster', 'UMAP_1', 'UMAP_2', 'patient_ID', 'condition'
    var: 'gene_ids', 'feature_types', 'gene_name', 'isoform1_name', 'isoform2_name'
    layers: 'ambiguous', 'isoform1', 'isoform2'

In [27]:
adata.obs['cluster'].unique()

['T cells', 'B cells', 'Myeloids', 'NK', 'Epithelial', 'Fibroblast']
Categories (6, object): ['B cells', 'Epithelial', 'Fibroblast', 'Myeloids', 'NK', 'T cells']

In [28]:
def get_celltype_adata(cellType):
    celltypeadata=adata[adata.obs['cluster']==cellType,:]
    celltype=cellType.replace(' ','_')
    file=celltype+'.h5ad'
    outputpath=os.path.join('/storage/yhhuang/users/ruiyan/NPC/diffcluster/',file)
    celltypeadata.write(outputpath)
    

In [29]:
for celltype in adata.obs['cluster'].unique():
    get_celltype_adata(celltype)

### get cdr file

In [30]:
import pandas as pd
import scanpy as sc
import numpy as np
import os

In [31]:
adata=sc.read('/storage/yhhuang/users/ruiyan/NPC/express_to_brie.h5ad')
adata

AnnData object with n_obs × n_vars = 51001 × 1784
    obs: 'cluster', 'UMAP_1', 'UMAP_2', 'patient_ID', 'condition'
    var: 'gene_ids', 'feature_types', 'gene_name', 'isoform1_name', 'isoform2_name'
    layers: 'ambiguous', 'isoform1', 'isoform2'

In [32]:
def get_cdr_file(cellcluster):
    cellcluster=cellcluster.replace(' ','_')
    inputfile='/storage/yhhuang/users/ruiyan/NPC/diffcluster/'+str(cellcluster)+'.h5ad'
    clusteradata=sc.read(inputfile)
    diseasedf=pd.DataFrame(clusteradata.obs['condition'])
    cdrdiseasedf = pd.get_dummies(diseasedf.condition, prefix='disease')
    
    cdr=np.array((clusteradata.X > 0).mean(1))[:,0]
    cdrdiseasedf['detect_rate']=cdr
    cdrdiseasedf.reset_index(inplace=True)
    

    cdrdiseasedf.drop(labels=['disease_NLH'],axis=1,inplace=True)
    print(cdrdiseasedf)
    outfile='/storage/yhhuang/users/ruiyan/NPC/diffcluster/NPC_cdr_'+cellcluster+'.tsv'
    cdrdiseasedf.to_csv(outfile,sep='\t',index=None)


In [33]:
for celltype in adata.obs['cluster'].unique():
    get_cdr_file(celltype)
    

                     index  disease_NPC  detect_rate
0      CACCACTAGGAGTCTG-11            0     0.174888
1       GACTAACGTCGGCTCA-9            1     0.135090
2       GCAGCCAGTAGCCTCG-5            1     0.043722
3       CACAAACGTCAGGACA-2            1     0.143498
4       TACGGTATCTCAACTT-1            1     0.108744
...                    ...          ...          ...
24566   CTAATGGGTGACTACT-1            1     0.147422
24567   CAAGAAAAGGCACATG-3            1     0.038117
24568  CACATTTGTGCTCTTC-10            1     0.186099
24569  CATGGCGAGAGCAATT-10            1     0.172646
24570   CTCGAGGGTTCATGGT-5            1     0.154148

[24571 rows x 3 columns]
                     index  disease_NPC  detect_rate
0       GCATGTAGTCTAGTCA-4            1     0.103139
1       TAAGAGAAGGAGTCTG-5            1     0.092489
2       CCGTACTCAGACAAAT-4            1     0.093610
3       TTGCCGTTCAACCATG-4            1     0.086323
4      GAGTCCGCATCGGTTA-12            0     0.246637
...                 

## Run Brie2

In [ ]:
#!/bin/bash

arr=(NK Myeloids Epithelial Fibroblast T_cells B_cells)
for i in ${arr[@]}
do
echao $i
brie-quant -i /storage/yhhuang/users/ruiyan/NPC/diffcluster/${i}.h5ad   -c /storage/yhhuang/users/ruiyan/NPC/diffcluster/NPC_cdr_${i}.tsv   -o /storage/yhhuang/users/ruiyan/NPC/diffcluster/Brie_out/brie_NPC_${i}.h5ad --batchSize 1000000 --minCell 10  --interceptMode gene --testBase full --LRTindex 0
done


# Do filtering (standard: FDR<0.01)

In [34]:
import pandas as pd
import scanpy as sc

In [35]:
def get_sign_gene(celltype):
    celltype=celltype.replace(' ','_')
    inputfile='/storage/yhhuang/users/ruiyan/NPC/diffcluster/Brie_out/brie_NPC_'+celltype+'.brie_ident.tsv'
    briedf=pd.read_csv(inputfile,delimiter='\t')
    print(briedf)
    signTdf=briedf[briedf['disease_NPC_FDR']<0.01]
    outputfile='/storage/yhhuang/users/ruiyan/NPC/diffcluster/sign_Gene/'+celltype+'.csv'
    signTdf.to_csv(outputfile,index=None)

    

In [36]:
adata=sc.read('/storage/yhhuang/users/ruiyan/NPC/srafiledir/scTSS_out/NPC_new_normalized.h5ad')
adata

AnnData object with n_obs × n_vars = 51001 × 4226
    obs: 'cluster', 'UMAP_1', 'UMAP_2', 'patient_ID', 'condition'
    var: 'TSS_start', 'TSS_end', 'gene_id', 'Chromosome', 'Feature', 'Start', 'End', 'Strand', 'gene_name', 'len'
    uns: 'log1p'
    obsm: 'X_umap'
    layers: 'counts'

In [37]:
adata.obs['cluster'].unique()


['T cells', 'B cells', 'Myeloids', 'NK', 'Epithelial', 'Fibroblast']
Categories (6, object): ['B cells', 'Epithelial', 'Fibroblast', 'Myeloids', 'NK', 'T cells']

In [38]:
for celltype in adata.obs['cluster'].unique():
    get_sign_gene(celltype)
    

               GeneID  n_counts  n_counts_uniq      cdr  intercept   sigma  \
0     ENSG00000000460       737            737  0.03345    -1.1680  2.4840   
1     ENSG00000000938       178            178  0.01604    -1.1560  1.7140   
2     ENSG00000001167       896            896  0.04786    -2.3100  1.3030   
3     ENSG00000001460        63             63  0.01701    -3.0630  1.6360   
4     ENSG00000001631      1919           1919  0.12460    -3.1440  0.7132   
...               ...       ...            ...      ...        ...     ...   
1623  ENSG00000278558       225            225  0.01457    -1.4710  1.9890   
1624  ENSG00000278709       325            325  0.01404    -1.6940  1.2780   
1625  ENSG00000284691        58             58  0.07920     0.7147  2.7520   
1626  ENSG00000285077       248            248  0.01095    -1.0360  1.2130   
1627  ENSG00000285106      2254           2254  0.25070    -3.4440  0.4510   

      disease_NPC_ceoff  disease_NPC_ELBO_gain  disease_NPC_pva